### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Parkinsons_Disease/GSE202667'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Step 1: Gene Expression Data Availability
is_gene_available = True  # As indicated by the series title, it mentions RNA indicating gene expression data.

# Step 2: Variable Availability and Data Type Conversion

# 2.1 Data Availability

# Parkinsons_Disease Availability
for key, values in {0: ["disease state: Parkinson's disease", 'disease state: Healthy Control']}.items():
    if len(set(values)) > 1:
        trait_row = key
        break

# Age Availability
for key, values in {3: ['age: 53', 'age: 57', 'age: 63', 'age: 75', 'age: 85', 'age: 76', 'age: 69', 'age: 66']}.items():
    if len(set(values)) > 1:
        age_row = key
        break

# Gender Availability
for key, values in {2: ['gender: male']}.items():
    if len(set(values)) > 1:
        gender_row = key
        break

# In this dataset, gender has only one value 'male', so we set it to None as per instructions.
if len(set({2: ['gender: male']}.get(2, []))) == 1:
    gender_row = None

# 2.3 Data Type Conversion

# Convert Parkinsons_Disease
def convert_trait(value):
    mapping = {
        "Parkinson's disease": 1,
        'Healthy Control': 0
    }
    value = value.split(':')[-1].strip()
    return mapping.get(value, None)

# Convert Age
def convert_age(value):
    try:
        value = value.split(':')[-1].strip()
        return float(value)
    except (ValueError, TypeError):
        return None

# Convert Gender
def convert_gender(value):
    mapping = {
        'male': 1,
        'female': 0
    }
    value = value.split(':')[-1].strip()
    return mapping.get(value, None)

# Step 3: Save Metadata
save_cohort_info('GSE202667', './preprocessed/Parkinsons_Disease/cohort_info.json', is_gene_available, trait_row is not None)

# Step 4: Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Parkinsons_Disease', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Parkinsons_Disease/trait_data/GSE202667.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:

# 1. Determine the identifier and gene symbol keys
identifier_key = 'SPOT_ID'
gene_symbol_key = 'GENE_SYMBOL'

# Verify that the necessary columns are present in the gene annotation dataframe
assert identifier_key in gene_annotation.columns, f"Identifier key '{identifier_key}' not found in gene annotation columns."
assert gene_symbol_key in gene_annotation.columns, f"Gene symbol key '{gene_symbol_key}' not found in gene annotation columns."

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# Debugging: Print the mapping dataframe to ensure the correct IDs are in place (Optional)
print("Mapping DataFrame Preview:")
print(preview_df(mapping_df))

# 3. Rename the gene expression data's index to match the identifier used in the mapping
gene_data.index.name = identifier_key  

# Apply the mapping and obtain the gene expression dataframe
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Print the resulting gene expression dataframe to ensure correctness (Optional)
print(preview_df(gene_data))


### Step 7: Data Normalization and Merging

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Parkinsons_Disease/gene_data/GSE202667.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Parkinsons_Disease')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE202667', './preprocessed/Parkinsons_Disease/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Parkinsons_Disease/GSE202667.csv'
    unbiased_merged_data.to_csv(csv_path)
